# Collision Avoidance - Live Demo
In this notebook we'll use the model we trained to detect whether the robot is free or blocked to enable a collision avoidance behavior on the robot. It is assumed that the optimized neural network has already been created and loaded into the Jupyter Lab file browser.

## TensorRT

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import cv2
import torch
from torchvision.models import resnet18
from torch2trt import TRTModule
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
import torchvision.transforms as transforms
import PIL.Image

# from utils import preprocess
import torch.nn.functional as F
import numpy as np
import time

WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


In [3]:
device = torch.device('cuda')

Load the optimized model by executing the cell below.

In [4]:
model_trt = TRTModule().to(device)
model_trt.load_state_dict(torch.load('best_model_resnet18_trt.pth', map_location=torch.device('cuda')))

<All keys matched successfully>

Inizialize `car` and `camera` objects.

In [5]:
car = NvidiaRacecar()
camera = CSICamera(width=224, height=224, capture_fps=65)

In [6]:
# device = torch.device('cuda')

In [7]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [8]:
print(torch.cuda.is_available())

True


## Live Demo

Due to the JetRacer's limited steering angle and obstacle visibility during the steering process, a specific time interval is required for successful obstacle avoidance. In the suggested demo, the robot will steer for 2 seconds upon obstacle detection. Nevertheless, this value should be dynamically modified to align with specific operational needs.

In [9]:
car.throttle = 0.0 
car.steering = 0.0

In [10]:
# car.throttle = 0.1
car.steering_gain = 1

In [ ]:
# import time
# import torch
# import torch.nn.functional as F

prob_blocked = 0
COLLISION_THRESHOLD = 0.92
AVOIDANCE_DURATION = 2.0  # Time to turn left (in seconds)
RETURN_DURATION = 1.5  # Time to turn right (in seconds)
STRAIGHT_DURATION = 1.0  # Time to go straight before resuming normal operation

# Possible states
NORMAL = 0
AVOIDING = 1
RETURNING = 2
STRAIGHTENING = 3

state = NORMAL
action_start_time = 0

while True:
    image = camera.read()
    image_proc = preprocess(image)
    output = model_trt(image_proc)
    y = F.softmax(output, dim=1)
    prob_blocked = float(y.flatten()[0])
    
    print(f"Collision probability: {prob_blocked:.4f}, State: {state}")
    
    current_time = time.time()
    
    if state == NORMAL:
        if prob_blocked > COLLISION_THRESHOLD:
            print(f"Obstacle detected! Probability: {prob_blocked:.4f}")
            car.steering = 0.9  # Turn left
            state = AVOIDING
            action_start_time = current_time
        else:
            car.steering = 0.0  # Go straight
    
    elif state == AVOIDING:
        if current_time - action_start_time >= AVOIDANCE_DURATION:
            print("Avoidance complete, returning to track")
            car.steering = -0.9  # Turn right
            state = RETURNING
            time.sleep(0.05)
            action_start_time = current_time
    
    elif state == RETURNING:
        if current_time - action_start_time >= RETURN_DURATION:
            print("Return complete, straightening")
            car.steering = 0.0  # Go straight
            state = STRAIGHTENING
            action_start_time = current_time
    
    elif state == STRAIGHTENING:
        if current_time - action_start_time >= STRAIGHT_DURATION:
            print("Resuming normal operation")
            state = NORMAL
    
    # Adjust throttle based on state
    if state == NORMAL:
        car.throttle = 0.2
    else:
        car.throttle = 0.2  # Slower speed during maneuvers
    
    # Optional: Add a small delay to control the loop rate
#     time.sleep(0.01)

In [11]:
from jetcam.utils import bgr8_to_jpeg
from IPython.display import display
import ipywidgets
import traitlets

widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)
blocked_slider = ipywidgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
# traitlets.dlink((camera, 'value'), (widget, 'value'), transform=bgr8_to_jpeg)


display(ipywidgets.HBox([widget, blocked_slider]))

In [12]:
car.throttle = 0.0
car.steering = 0.0

In [13]:
prob_blocked = 0
threshold = 0.5

turning = False  # Flag to indicate if the car is turning
turn_time = time.time()  # Start a timer
# print("turn time", turn_time)
car.throttle = 0.3
def update(change):
    global blocked_slider, turning
    image = change['new']
#     image = camera.read()
    image_proc = preprocess(image)
    output = model_trt(image_proc)
    y = F.softmax(output, dim=1)
    prob_blocked = float(y.flatten()[0])
    widget.value = bgr8_to_jpeg(image)
    blocked_slider.value = prob_blocked
#     print(prob_blocked)

    
    if  prob_blocked> 0.7 and not turning:
        car.throttle = 0.1
        car.steering = -0.6
#         time.sleep(0.1)
        turning = True

    if turning:
        car.throttle = 0.0
#         car.steering = 1
        
        
#         car.throttle = 0.1
#     elif  prob_blocked >0.92:
#         car.throttle = 0.0
# #         break
#     else :
#         print("===========", str(prob_blocked))
#         car.throttle = 0.0

#     elif prob_blocked > threshold and not turning:
#         car.steering = 0.9  # Choose a steering angle
#         turning = True  # Set the turning flag
#         turn_time = time.time()  # Start a timer

#     if prob_blocked < threshold and turning:
#         if time.time() - turn_time >= 2:
#             car.steering_gain = 1
#             car.steering = -0.9  # Straighten the wheels
#             turning = False  # Clear the turning flag
    
#     car.throttle = 0.3  # Choose car.throttle
    time.sleep(0.001)
update({'new': camera.value})


In [14]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera


In [15]:
camera.running = True